In [1]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd


from pydataset import data

import acquire
import prepare

Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

In [2]:
titanic = prepare.prep_titanic()
titanic

,passenger_id,survived,pclass,sibsp,parch,fare,alone,sex_male,embark_town_Queenstown,embark_town_Southampton
0,0,0,3,1,0,7.2500,0,1,0,1
1,1,1,1,1,0,71.2833,0,0,0,0
2,2,1,3,0,0,7.9250,1,0,0,1
3,3,1,1,1,0,53.1000,0,0,0,1
4,4,0,3,0,0,8.0500,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...
886,886,0,2,0,0,13.0000,1,1,0,1
887,887,1,1,0,0,30.0000,1,0,0,1
888,888,0,3,1,2,23.4500,0,0,0,1
889,889,1,1,0,0,30.0000,1,1,0,0


In [16]:
baseline = 549 / (549 + 342)
baseline

0.6161616161616161

In [4]:
train, validate, test = prepare.split_data(titanic, 'survived')

In [6]:
X_train = train.drop(columns=['survived'])
y_train = train.survived

X_validate = validate.drop(columns=['survived'])
y_validate = validate.survived

X_test = test.drop(columns=['survived'])
y_test = test.survived

In [9]:
rf = RandomForestClassifier(bootstrap=True,
                            class_weight=None,
                            criterion='gini',
                            min_samples_leaf=1,
                            n_estimators=100,
                            max_depth=3,
                            random_state=823)

In [10]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, n_estimators=10, random_state=823)

In [11]:
print(rf.feature_importances_)

[0.02553564 0.06588117 0.09225177 0.04731021 0.25966947 0.0199035
 0.48407932 0.         0.00536893]


In [15]:
y_pred = rf.predict(X_train)
y_pred[0:10]

array([0, 1, 1, 0, 1, 0, 1, 1, 0, 0])

In [18]:
y_pred_proba = rf.predict_proba(X_train)
y_pred_proba

array([[0.87971185, 0.12028815],
       [0.24323142, 0.75676858],
       [0.15416097, 0.84583903],
       ...,
       [0.52943464, 0.47056536],
       [0.45916159, 0.54083841],
       [0.24056475, 0.75943525]])

In [ ]:
Evaluate your results using the model score, confusion matrix, and classification report.

In [19]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.80


In [20]:
print(confusion_matrix(y_train, y_pred))

[[305  24]
 [ 84 121]]


In [21]:
labels = sorted(y_train.unique())
pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels, columns=labels)

,0,1
0,305,24
1,84,121


In [22]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.93      0.85       329
           1       0.83      0.59      0.69       205

    accuracy                           0.80       534
   macro avg       0.81      0.76      0.77       534
weighted avg       0.80      0.80      0.79       534



In [ ]:
Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
Run through steps increasing your min_samples_leaf and decreasing your max_depth.

In [84]:
rf2 = RandomForestClassifier(bootstrap=True,
                            class_weight=None,
                            criterion='gini',
                            min_samples_leaf=5,
                            n_estimators=10,
                            max_depth=2,
                            random_state=823)

In [85]:
rf2.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, min_samples_leaf=5, n_estimators=10,
                       random_state=823)

In [86]:
print(rf2.feature_importances_)
y_pred = rf2.predict(X_train)
y_pred[0:10]

[0.         0.08816828 0.12458945 0.07085037 0.24414779 0.02273863
 0.44713566 0.         0.00236981]


array([0, 1, 1, 0, 1, 0, 1, 1, 0, 0])

In [87]:
y_pred_proba = rf2.predict_proba(X_train)
y_pred_proba

array([[0.82833001, 0.17166999],
       [0.34887232, 0.65112768],
       [0.29148988, 0.70851012],
       ...,
       [0.50489954, 0.49510046],
       [0.40200401, 0.59799599],
       [0.34887232, 0.65112768]])

In [88]:
print('Accuracy of random forest classifier on training set: {:.2f}'
     .format(rf2.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.80


In [89]:
print(confusion_matrix(y_train, y_pred))

[[305  24]
 [ 85 120]]


In [90]:
labels = sorted(y_train.unique())
pd.DataFrame(confusion_matrix(y_train, y_pred), index=labels, columns=labels)

,0,1
0,305,24
1,85,120


In [91]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.93      0.85       329
           1       0.83      0.59      0.69       205

    accuracy                           0.80       534
   macro avg       0.81      0.76      0.77       534
weighted avg       0.80      0.80      0.79       534



In [92]:
What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

Object `Why` not found.


In [ ]:
After making a few models, which one has the best performance (or closest metrics) on both train and validate?